<a href="https://colab.research.google.com/github/MENAKAANBUKKARASU/kitchen_Review_model/blob/main/food_kitchen_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import make_scorer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

# Sample data

In [ ]:
reviews = [
    "Praises about the quality",
    "Praises about Timeliness",
    "Complaints about Quantity",
    "Really impressed with the high-quality service and timely delivery.",
    "Not satisfied with the quantity of food provided.",
    "Great quality, but the delivery was a bit late.",
    "Service was quick, and the food exceeded my expectations.",
    "Quantity of the order was disappointing. Won't order again.",
    "Amazing food, delivered on time. Highly recommended."
]
sentiments = ["positive", "positive", "negative", "positive", "negative", "positive", "positive", "negative", "positive"]


In [ ]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(reviews, sentiments, test_size=0.2, random_state=42)

In [ ]:
# Upsample the minority class (negative sentiment)
X_train_positive = [X_train[i] for i in range(len(X_train)) if y_train[i] == 'positive']
X_train_negative = [X_train[i] for i in range(len(X_train)) if y_train[i] == 'negative']

X_train_negative_upsampled = resample(X_train_negative, replace=True, n_samples=len(X_train_positive), random_state=42)
X_train_upsampled = X_train_positive + X_train_negative_upsampled
y_train_upsampled = ['positive'] * len(X_train_positive) + ['negative'] * len(X_train_negative_upsampled)

In [ ]:
# Create a pipeline with TF-IDF vectorizer and Naive Bayes classifier
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
    ('clf', MultinomialNB())
])

In [ ]:
# Define a parameter grid for GridSearchCV
param_grid = {
    'tfidf__max_features': [1000, 2000, 3000],
    'clf__alpha': [0.1, 0.5, 1.0],
}

In [ ]:
# Use GridSearchCV to find the best parameters
grid_search = GridSearchCV(text_clf, param_grid, cv=5, scoring=make_scorer(accuracy_score))
grid_search.fit(X_train_upsampled, y_train_upsampled)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(ngram_range=(1, 2))),
                                       ('clf', MultinomialNB())]),
             param_grid={'clf__alpha': [0.1, 0.5, 1.0],
                         'tfidf__max_features': [1000, 2000, 3000]},
             scoring=make_scorer(accuracy_score))

In [ ]:
# Print the best parameters
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'clf__alpha': 0.1, 'tfidf__max_features': 1000}


In [ ]:
# Predicting on the test set
predictions = grid_search.predict(X_test)

In [ ]:
# Evaluating the model
print("Accuracy:", accuracy_score(y_test, predictions))
print("Classification Report:\n", classification_report(y_test, predictions))

Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

    negative       1.00      1.00      1.00         1
    positive       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



In [ ]:
!pip install sumy

In [ ]:
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.7 MB/s eta 0:00:00


**SENTIMENT DEFAULTLY INCLUDED**

In [ ]:
import nltk

# Download the 'punkt' tokenizer data
nltk.download('punkt')

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.utils import resample

def generate_summary(reviews):
    # Concatenate all reviews into a single document
    document = "\n".join(reviews)

    # Use sumy to parse the document and apply TextRank summarization
    parser = PlaintextParser.from_string(document, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    summary = summarizer(parser.document, sentences_count=3)  # Adjust sentences_count as needed

    # Convert the summary back to a string
    summary_text = " ".join(str(sentence) for sentence in summary)

    return summary_text

# Sample data
reviews = [
    "Praises about the quality",
    "Praises about Timeliness",
    "Complaints about Quantity",
    "Really impressed with the high-quality service and timely delivery.",
    "Not satisfied with the quantity of food provided.",
    "Great quality, but the delivery was a bit late.",
    "Service was quick, and the food exceeded my expectations.",
    "Quantity of the order was disappointing. Won't order again.",
    "Amazing food, delivered on time. Highly recommended.",
    "The food quality was inconsistent. Some dishes were excellent, while others were mediocre.",
    "Delivery was prompt for the most part, but there was a slight delay in one of my orders.",
    "Portion sizes are generous, ensuring a satisfying meal. Appreciate the attention to quantity.",
    "Packaging was reliable, keeping the food in good condition. Eco-friendly materials are a plus.",
    "Overall, satisfied with the orders. However, improvements in consistency would elevate the experience."
]
sentiments = ["positive", "positive", "negative", "positive", "negative", "positive", "positive", "negative", "positive", "neutral", "neutral", "positive", "neutral", "positive"]
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(reviews, sentiments, test_size=0.2, random_state=42)

# Upsample the minority class (negative sentiment)
X_train_positive = [X_train[i] for i in range(len(X_train)) if y_train[i] == 'positive']
X_train_negative = [X_train[i] for i in range(len(X_train)) if y_train[i] == 'negative']

X_train_negative_upsampled = resample(X_train_negative, replace=True, n_samples=len(X_train_positive), random_state=42)
X_train_upsampled = X_train_positive + X_train_negative_upsampled
y_train_upsampled = ['positive'] * len(X_train_positive) + ['negative'] * len(X_train_negative_upsampled)

# Create a pipeline with TF-IDF vectorizer and Naive Bayes classifier
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
    ('clf', MultinomialNB())
])

# Fit the model on the upsampled training data
text_clf.fit(X_train_upsampled, y_train_upsampled)

# Predicting on all reviews
all_predictions = text_clf.predict(reviews)

# Group reviews by sentiment and factor
positive_quality_reviews = [reviews[i] for i in range(len(reviews)) if all_predictions[i] == 'positive' and 'quality' in reviews[i].lower()]
neutral_quality_reviews = [reviews[i] for i in range(len(reviews)) if all_predictions[i] == 'neutral' and 'quality' in reviews[i].lower()]
negative_quality_reviews = [reviews[i] for i in range(len(reviews)) if all_predictions[i] == 'negative' and 'quality' in reviews[i].lower()]

# Function for in-line summarization
def in_line_summary(reviews, factor):
    factor_reviews = [review for review in reviews if factor.lower() in review.lower()]
    factor_summary = generate_summary(factor_reviews)
    return factor_summary

# Generate summaries for each factor and sentiment category
positive_quality_summary = in_line_summary(positive_quality_reviews, 'quality')
neutral_quality_summary = in_line_summary(neutral_quality_reviews, 'quality')
negative_quality_summary = in_line_summary(negative_quality_reviews, 'quality')

# Print summaries
print("Positive Quality Summary:\n", positive_quality_summary)
print("\nNeutral Quality Summary:\n", neutral_quality_summary)
print("\nNegative Quality Summary:\n", negative_quality_summary)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Positive Quality Summary:
 Praises about the quality Really impressed with the high-quality service and timely delivery. Great quality, but the delivery was a bit late. Some dishes were excellent, while others were mediocre.

Neutral Quality Summary:
 

Negative Quality Summary:
 


In [ ]:
import nltk
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.utils import resample

# Download the 'punkt' tokenizer data
nltk.download('punkt')

def in_line_summary(reviews):
    # Concatenate all reviews into a single document
    document = "\n".join(reviews)

    # Use sumy to generate a summary for all reviews
    parser = PlaintextParser.from_string(document, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    overall_summary = summarizer(parser.document, sentences_count=3)  # Adjust sentences_count as needed

    # Convert the summary back to a string
    overall_summary_text = " ".join(str(sentence) for sentence in overall_summary)

    return overall_summary_text

# Sample data
reviews = [
    "Praises about the quality.",
    "Really impressed with the high-quality service and timely delivery.",
    "Service was quick, and the food exceeded my expectations.",
    "Amazing food, delivered on time. Highly recommended.",
    "Portion sizes are generous, ensuring a satisfying meal. Appreciate the attention to quantity.",
    "Packaging was reliable, keeping the food in good condition. Eco-friendly materials are a plus.",
    "Overall, satisfied with the orders. However, improvements in consistency would elevate the experience.",
    "While the food is generally good, with some dishes standing out more than others. While flavors are present, there is room for improvement in ensuring a more consistent and elevated standard across the entire menu.",
    "Packaging is generally sufficient, though there is room for improvement. On occasion, items have arrived slightly disheveled, suggesting that tighter seals and sturdier materials could enhance the overall delivery experience.",
    "Complaints about Quantity.",
    "Not satisfied with the quantity of food provided.",
    "Quantity of the order was disappointing. Won't order again.",
    "The food quality was inconsistent. Some dishes were excellent, while others were mediocre.",
    "Critical Review: Inconsistency in quality is a concern. While some dishes are impressive, others fall short, indicating a lack of uniformity in the preparation. Addressing this issue would greatly enhance the overall dining experience.",
    "Delivery was prompt for the most part, but there was a slight delay in one of my orders."
]
sentiments = [
    "positive",
    "positive",
    "positive",
    "positive",
    "positive",
    "positive",
    "positive",
    "neutral",
    "neutral",
    "negative",
    "negative",
    "negative",
    "negative",
    "negative",
    "positive",
]

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(reviews, sentiments, test_size=0.2, random_state=42)

# Upsample the minority class (negative sentiment)
X_train_positive = [X_train[i] for i in range(len(X_train)) if y_train[i] == 'positive']
X_train_negative = [X_train[i] for i in range(len(X_train)) if y_train[i] == 'negative']

X_train_negative_upsampled = resample(X_train_negative, replace=True, n_samples=len(X_train_positive), random_state=42)
X_train_upsampled = X_train_positive + X_train_negative_upsampled
y_train_upsampled = ['positive'] * len(X_train_positive) + ['negative'] * len(X_train_negative_upsampled)

# Create a pipeline with TF-IDF vectorizer and Naive Bayes classifier
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
    ('clf', MultinomialNB())
])

# Fit the model on the upsampled training data
text_clf.fit(X_train_upsampled, y_train_upsampled)

# Predicting on all reviews
all_predictions = text_clf.predict(reviews)

# Generate an overall summary for all reviews
overall_summary = in_line_summary(reviews)

# Print the overall summary
print("Overall Summary:\n", overall_summary)


Overall Summary:
 While flavors are present, there is room for improvement in ensuring a more consistent and elevated standard across the entire menu. While some dishes are impressive, others fall short, indicating a lack of uniformity in the preparation. Delivery was prompt for the most part, but there was a slight delay in one of my orders.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

def generate_overall_summary(reviews):
    # Concatenate all reviews into a single document
    document = "\n".join(reviews)

    # Use sumy to parse the document and apply TextRank summarization
    parser = PlaintextParser.from_string(document, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    overall_summary = summarizer(parser.document, sentences_count=3)  # Adjust sentences_count as needed

    # Convert the overall summary back to a string
    overall_summary_text = " ".join(str(sentence) for sentence in overall_summary)

    return overall_summary_text

# Sample data
reviews = [
    "Praises about the quality.",
    "Really impressed with the high-quality service and timely delivery.",
    "Service was quick, and the food exceeded my expectations.",
    "Amazing food, delivered on time. Highly recommended.",
    "Portion sizes are generous, ensuring a satisfying meal. Appreciate the attention to quantity.",
    "Packaging was reliable, keeping the food in good condition. Eco-friendly materials are a plus.",
    "Overall, satisfied with the orders. However, improvements in consistency would elevate the experience.",
    "While the food is generally good, with some dishes standing out more than others. While flavors are present, there is room for improvement in ensuring a more consistent and elevated standard across the entire menu.",
    "Packaging is generally sufficient, though there is room for improvement. On occasion, items have arrived slightly disheveled, suggesting that tighter seals and sturdier materials could enhance the overall delivery experience.",
    "Complaints about Quantity.",
    "Not satisfied with the quantity of food provided.",
    "Quantity of the order was disappointing. Won't order again.",
    "The food quality was inconsistent. Some dishes were excellent, while others were mediocre.",
    "Critical Review: Inconsistency in quality is a concern. While some dishes are impressive, others fall short, indicating a lack of uniformity in the preparation. Addressing this issue would greatly enhance the overall dining experience.",
    "Delivery was prompt for the most part, but there was a slight delay in one of my orders."
]

# Generate the overall summary
overall_summary = generate_overall_summary(reviews)

# Print the overall summary
print("Overall Summary:\n", overall_summary)


Overall Summary:
 While flavors are present, there is room for improvement in ensuring a more consistent and elevated standard across the entire menu. While some dishes are impressive, others fall short, indicating a lack of uniformity in the preparation. Delivery was prompt for the most part, but there was a slight delay in one of my orders.


In [ ]:
import nltk
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.utils import resample
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer  # Import SentimentIntensityAnalyzer from vaderSentiment

# Download the 'punkt' tokenizer data
nltk.download('punkt')

# Function to perform sentiment analysis on a review
def analyze_sentiment(review):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = analyzer.polarity_scores(review)

    # Classify the sentiment based on compound score
    if sentiment_scores['compound'] >= 0.05:
        return 'positive'
    elif sentiment_scores['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Function to generate overall summary with sentiment and aspect-based analysis
def generate_overall_summary(reviews):
    # Separate reviews into positive, neutral, and negative categories
    positive_reviews = [review for review in reviews if analyze_sentiment(review) == 'positive']
    neutral_reviews = [review for review in reviews if analyze_sentiment(review) == 'neutral']
    negative_reviews = [review for review in reviews if analyze_sentiment(review) == 'negative']

    # Generate aspect-based summaries for each sentiment category
    positive_summary = in_line_summary(positive_reviews)
    neutral_summary = in_line_summary(neutral_reviews)
    negative_summary = in_line_summary(negative_reviews)

    # Combine aspect-based summaries into an overall summary
    overall_summary = f"Positive Reviews Summary:\n{positive_summary}\n\nNeutral Reviews Summary:\n{neutral_summary}\n\nNegative Reviews Summary:\n{negative_summary}"

    return overall_summary

# Sample data
reviews = [
    "Praises about the quality.",
    "Really impressed with the high-quality service and timely delivery.",
    "Service was quick, and the food exceeded my expectations.",
    "Amazing food, delivered on time. Highly recommended.",
    "Portion sizes are generous, ensuring a satisfying meal. Appreciate the attention to quantity.",
    "Packaging was reliable, keeping the food in good condition. Eco-friendly materials are a plus.",
    "Overall, satisfied with the orders. However, improvements in consistency would elevate the experience.",
    "While the food is generally good, with some dishes standing out more than others. While flavors are present, there is room for improvement in ensuring a more consistent and elevated standard across the entire menu.",
    "Packaging is generally sufficient, though there is room for improvement. On occasion, items have arrived slightly disheveled, suggesting that tighter seals and sturdier materials could enhance the overall delivery experience.",
    "Complaints about Quantity.",
    "Not satisfied with the quantity of food provided.",
    "Quantity of the order was disappointing. Won't order again.",
    "The food quality was inconsistent. Some dishes were excellent, while others were mediocre.",
    "Critical Review: Inconsistency in quality is a concern. While some dishes are impressive, others fall short, indicating a lack of uniformity in the preparation. Addressing this issue would greatly enhance the overall dining experience.",
    "Delivery was prompt for the most part, but there was a slight delay in one of my orders."
]

# Generate the overall summary with sentiment and aspect-based analysis
overall_summary = generate_overall_summary(reviews)

# Print the overall summary
print("Overall Summary with Sentiment and Aspect-Based Analysis:\n", overall_summary)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Overall Summary with Sentiment and Aspect-Based Analysis:
 Positive Reviews Summary:
While the food is generally good, with some dishes standing out more than others. While flavors are present, there is room for improvement in ensuring a more consistent and elevated standard across the entire menu. On occasion, items have arrived slightly disheveled, suggesting that tighter seals and sturdier materials could enhance the overall delivery experience.

Neutral Reviews Summary:
Service was quick, and the food exceeded my expectations.

Negative Reviews Summary:
Quantity of the order was disappointing. While some dishes are impressive, others fall short, indicating a lack of uniformity in the preparation. Delivery was prompt for the most part, but there was a slight delay in one of my orders.


In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer  # Import SentimentIntensityAnalyzer from vaderSentiment

# Function to perform sentiment analysis on a review
def analyze_sentiment(review):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = analyzer.polarity_scores(review)

    # Return the compound score
    return sentiment_scores['compound']

# Function for in-line summarization
def in_line_summary(reviews, aspect):
    aspect_reviews = [review for review in reviews if aspect.lower() in review.lower()]
    if not aspect_reviews:
        return ""

    # Concatenate aspect-related reviews into a single document
    document = "\n".join(aspect_reviews)

    # Use sumy to parse the document and apply TextRank summarization
    parser = PlaintextParser.from_string(document, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    aspect_summary = summarizer(parser.document, sentences_count=2)  # Adjust sentences_count as needed

    # Convert the summary back to a string
    aspect_summary_text = " ".join(str(sentence) for sentence in aspect_summary)

    return aspect_summary_text

# Function to generate star rating for the kitchen based on reviews
def generate_star_rating(reviews):
    # Calculate the average compound score for all reviews
    average_compound_score = sum(analyze_sentiment(review) for review in reviews) / len(reviews)

    # Generate aspect-based summaries for different aspects
    quality_summary = in_line_summary(reviews, 'quality')
    quantity_summary = in_line_summary(reviews, 'quantity')
    package_summary = in_line_summary(reviews, 'package')
    timeliness_summary = in_line_summary(reviews, 'timeliness')
    hygiene_summary = in_line_summary(reviews, 'consistent hygiene standards')
    value_summary = in_line_summary(reviews, 'value for money')

    # Combine aspect-based summaries into an overall summary
    overall_summary = f"Quality Summary:\n{quality_summary}\n\nQuantity Summary:\n{quantity_summary}\n\nPackage Summary:\n{package_summary}\n\nTimeliness Summary:\n{timeliness_summary}\n\nHygiene Summary:\n{hygiene_summary}\n\nValue Summary:\n{value_summary}"

    # Assign stars based on the modified compound score
    if average_compound_score >= 0.8:
        stars = 5  # Best, assign 5 stars
    elif average_compound_score >= 0.4:
        stars = 4  # More than average, assign 4 stars
    elif average_compound_score >= 0.25:
        stars = 3  # Neutral, assign 3 stars
    elif average_compound_score >= -0.3:
        stars = 2  # Below average, assign 2 stars
    else:
        stars = 1  # Critical, assign 1 star

    return stars, overall_summary

# Sample data
reviews = [
    "   - Every dish reflects a commitment to using high-quality, fresh ingredients.",
    "   - Flavors are consistently rich, showcasing a genuine passion for culinary excellence.",

    "   - Impressive variety in the menu, catering to different tastes and preferences.",
    "   - Regular updates to the menu keep the dining experience exciting and diverse.",

    "   - Reliable and timely deliveries ensure that the food arrives fresh and warm.",
    "   - Efficient service, adhering to promised delivery times consistently.",

    "   - Portions are not only ample but also well-balanced, providing great value for money.",
    "   - The kitchen understands the importance of satisfying portion sizes for a fulfilling meal.",

    "   - Impeccable packaging that keeps the food in perfect condition during transit.",
    "   - Use of eco-friendly materials reflects a commitment to sustainability.",

    "   - Consistency in maintaining a high standard across the entire menu.",
    "   - Each visit or order guarantees the same exceptional quality.",

    "   - Innovative and creative approach to recipes sets this kitchen apart.",
    "   - The chef's ability to introduce unique and delightful flavors is commendable.",

    "   - Clear communication regarding orders and any potential delays.",
    "   - Friendly and responsive customer service that addresses inquiries promptly.",

    "   - Attention to detail in the presentation of each dish enhances the overall dining experience.",
    "   - Dishes are visually appealing, showcasing the kitchen's dedication to aesthetics.",

    "   - Pricing is reasonable, considering the quality and quantity of the offerings.",
    "   - Every penny spent feels justified given the delightful dining experience.",

    "   - Willingness to accommodate special requests or dietary restrictions.",
    "   - Customization options allow customers to tailor their orders to personal preferences.",

    "   - Active engagement with the local community, contributing to a sense of belonging.",
    "   - Involvement in community events or initiatives demonstrates a commitment beyond just food.",

    "   - Strict adherence to hygiene and safety protocols.",
    "   - Customers feel assured about the cleanliness and safety of the kitchen.",

    "   - Seamless and user-friendly online ordering system.",
    "   - Quick and hassle-free ordering process, enhancing overall convenience.",

    "   - Occasionally includes surprise treats or samples, adding a delightful touch.",
    "   - The element of surprise creates a positive and memorable customer experience.",

    "   - Clear information about ingredients used, catering to those with specific dietary concerns.",
    "   - Transparency builds trust and loyalty among customers.",

    "   - Offers special packages or customized menus for events and celebrations.",
    "   - The kitchen goes the extra mile to make special occasions memorable.",

    "   - Actively incorporates customer feedback into menu improvements.",
    "   - Demonstrates a commitment to evolving based on customer preferences.",

    "   - Actively involved in sustainable practices, such as minimizing food waste.",
    "   - Strives to minimize the environmental impact through eco-friendly initiatives.",

    "   - Consistently positive experiences contribute to long-term customer loyalty.",
    "   - The home kitchen is a reliable go-to for quality meals, creating a loyal customer base."
]

# Generate star rating and overall summary
stars, overall_summary = generate_star_rating(reviews)

# Print the results
print("Star Rating:", stars)
print("\nOverall Summary:\n", overall_summary)


Star Rating: 4

Overall Summary:
 Quality Summary:
- Pricing is reasonable, considering the quality and quantity of the offerings. - The home kitchen is a reliable go-to for quality meals, creating a loyal customer base.

Quantity Summary:
- Pricing is reasonable, considering the quality and quantity of the offerings.

Package Summary:
- Offers special packages or customized menus for events and celebrations.

Timeliness Summary:


Hygiene Summary:


Value Summary:
- Portions are not only ample but also well-balanced, providing great value for money.


data set included code

In [ ]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Function to perform sentiment analysis on a review
def analyze_sentiment(review):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = analyzer.polarity_scores(review)

    # Return the compound score
    return sentiment_scores['compound']

# Function for in-line summarization considering sentiment
def in_line_summary_sentiment(reviews, aspect, sentiment):
    aspect_reviews = [review for review in reviews if aspect.lower() in review.lower() and analyze_sentiment(review) == sentiment]
    if not aspect_reviews:
        return f"No comments available for {aspect.capitalize()} in the {sentiment} sentiment."

    # Concatenate aspect-related reviews into a single document
    document = "\n".join(aspect_reviews)

    # Use sumy to parse the document and apply TextRank summarization
    parser = PlaintextParser.from_string(document, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    aspect_summary = summarizer(parser.document, sentences_count=4)  # Adjust sentences_count as needed

    # Convert the summary back to a string
    aspect_summary_text = " ".join(str(sentence) for sentence in aspect_summary)

    return aspect_summary_text

# Function to generate star rating for the kitchen based on reviews considering sentiment
def generate_star_rating_sentiment(reviews):
    # Calculate the average compound score for all reviews
    average_compound_score = sum(analyze_sentiment(review) for review in reviews) / len(reviews)

    # Generate aspect-based summaries for different aspects and sentiments
    quality_summary_pos = in_line_summary_sentiment(reviews, 'quality', 'positive')
    quality_summary_neg = in_line_summary_sentiment(reviews, 'quality', 'negative')

    quantity_summary_pos = in_line_summary_sentiment(reviews, 'quantity', 'positive')
    quantity_summary_neg = in_line_summary_sentiment(reviews, 'quantity', 'negative')

    package_summary_pos = in_line_summary_sentiment(reviews, 'package', 'positive')
    package_summary_neg = in_line_summary_sentiment(reviews, 'package', 'negative')

    timeliness_summary_pos = in_line_summary_sentiment(reviews, 'timeliness', 'positive')
    timeliness_summary_neg = in_line_summary_sentiment(reviews, 'timeliness', 'negative')

    # Combine aspect-based summaries into an overall summary
    overall_summary = f"\nQuality Summary (Positive Sentiment):\n{quality_summary_pos}\nQuality Summary (Negative Sentiment):\n{quality_summary_neg}\n\nQuantity Summary (Positive Sentiment):\n{quantity_summary_pos}\nQuantity Summary (Negative Sentiment):\n{quantity_summary_neg}\n\nPackage Summary (Positive Sentiment):\n{package_summary_pos}\nPackage Summary (Negative Sentiment):\n{package_summary_neg}\n\nTimeliness Summary (Positive Sentiment):\n{timeliness_summary_pos}\nTimeliness Summary (Negative Sentiment):\n{timeliness_summary_neg}"

    # Assign stars based on the average compound score
    if average_compound_score >= 0.5:
        stars = 5  # Best, assign 5 stars
    elif average_compound_score >= 0.4:
        stars = 4  # More than average, assign 4 stars
    elif average_compound_score >= 0.1:
        stars = 3  # Neutral, assign 3 stars
    elif average_compound_score >= -0.3:
        stars = 2  # Below average, assign 2 stars
    else:
        stars = 1  # Critical, assign 1 star

    return stars, overall_summary

# Function to extract important words from reviews
def extract_important_words(reviews):
    # Create a CountVectorizer with English stop words
    vectorizer = CountVectorizer(stop_words='english')  # Set stop_words to 'english'

    # Fit and transform the reviews
    X = vectorizer.fit_transform(reviews)

    # Get the feature names (words)
    feature_names = vectorizer.get_feature_names_out()

    # Sum the counts of each word
    word_counts = X.sum(axis=0).A1

    # Create a dictionary with words and their counts
    words_and_counts = dict(zip(feature_names, word_counts))

    # Sort the dictionary by counts in descending order
    sorted_words_and_counts = sorted(words_and_counts.items(), key=lambda x: x[1], reverse=True)

    # Extract the top 10 important words
    top_10_words = [word for word, count in sorted_words_and_counts[:10]]

    return top_10_words

# Load data from CSV file
df = pd.read_csv('Restaurant_Reviews.csv')  # Replace 'your_file.csv' with the actual file name

# Iterate through each kitchen in the DataFrame
for kitchen_name, group_df in df.groupby('Kitchen'):
    # Extract reviews for the current kitchen
    reviews = group_df['Review'].tolist()

    # Generate star rating and overall summary considering sentiment for the current kitchen
    stars, overall_summary = generate_star_rating_sentiment(reviews)

    # Extract important words from reviews
    top_10_words = extract_important_words(reviews)

    # Print the results for the current kitchen
    print(f"\nKitchen Name: {kitchen_name}")
    print("Star Rating:", stars)
    print("\nOverall Summary:\n", overall_summary)
    print("\nTop 10 Important Words:", top_10_words)
    print("\n" + "="*50 + "\n")



Kitchen Name: ANU'S KITCHEN
Star Rating: 3

Overall Summary:
 
Quality Summary (Positive Sentiment):
No comments available for Quality in the positive sentiment.
Quality Summary (Negative Sentiment):
No comments available for Quality in the negative sentiment.

Quantity Summary (Positive Sentiment):
No comments available for Quantity in the positive sentiment.
Quantity Summary (Negative Sentiment):
No comments available for Quantity in the negative sentiment.

Package Summary (Positive Sentiment):
No comments available for Package in the positive sentiment.
Package Summary (Negative Sentiment):
No comments available for Package in the negative sentiment.

Timeliness Summary (Positive Sentiment):
No comments available for Timeliness in the positive sentiment.
Timeliness Summary (Negative Sentiment):
No comments available for Timeliness in the negative sentiment.

Top 10 Important Words: ['like', 'food', 'good', 'great', 'place', 'service', 'just', 'time', 'amazing', 'deal']



Kitchen 

In [ ]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

# Function to perform sentiment analysis on a review
def analyze_sentiment(review):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = analyzer.polarity_scores(review)
    # Return the compound score
    return sentiment_scores['compound']

# Function for in-line summarization
def generate_summary(reviews):
    # Concatenate reviews into a single document
    document = "\n".join(reviews)
    # Use sumy to parse the document and apply TextRank summarization
    parser = PlaintextParser.from_string(document, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    # Generate summary of 2 sentences
    summary = summarizer(parser.document, sentences_count=6)
    # Convert the summary back to a string
    summary_text = " ".join(str(sentence) for sentence in summary)
    return summary_text

# Function to generate aspect-based star rating for quality, quantity, and timeliness
def generate_aspect_star_rating(reviews, aspect):
    # Calculate the average compound score for reviews mentioning the aspect
    aspect_reviews = [review for review in reviews if aspect.lower() in review.lower()]
    if not aspect_reviews:
        return 0
    aspect_compound_scores = [analyze_sentiment(review) for review in aspect_reviews]
    aspect_average_score = sum(aspect_compound_scores) / len(aspect_compound_scores)
    # Assign stars based on the average compound score
    if aspect_average_score >= 0.5:
        stars = 5  # Best, assign 5 stars
    elif aspect_average_score >= 0.4:
        stars = 4  # More than average, assign 4 stars
    elif aspect_average_score >= 0.1:
        stars = 3  # Neutral, assign 3 stars
    elif aspect_average_score >= -0.3:
        stars = 2  # Below average, assign 2 stars
    else:
        stars = 1  # Critical, assign 1 star
    return stars

# Load the CSV file into a DataFrame
df = pd.read_csv("Restaurant_Reviews.csv")

# Iterate through each kitchen in the DataFrame
for kitchen_name, group_df in df.groupby('Kitchen'):
    # Extract reviews for the current kitchen
    reviews = group_df['Review'].tolist()

    # Generate aspect-based star ratings and summaries
    quality_stars = generate_aspect_star_rating(reviews, 'quality')
    quantity_stars = generate_aspect_star_rating(reviews, 'quantity')
    timeliness_stars = generate_aspect_star_rating(reviews, 'timeliness')
    quality_summary = generate_summary([review for review in reviews if 'quality' in review.lower()])
    quantity_summary = generate_summary([review for review in reviews if 'quantity' in review.lower()])
    timeliness_summary = generate_summary([review for review in reviews if 'timeliness' in review.lower()])

    # Print the results for the current kitchen
    print(f"\nKitchen Name: {kitchen_name}")
    print("Quality Star Rating:", quality_stars)
    print("Quality Summary:", quality_summary)
    print("Quantity Star Rating:", quantity_stars)
    print("Quantity Summary:", quantity_summary)
    print("Timeliness Star Rating:", timeliness_stars)
    print("Timeliness Summary:", timeliness_summary)
    print("\n" + "="*50 + "\n")


Kitchen Name: ANU'S KITCHEN
Quality Star Rating: 2
Quality Summary: The only redeeming quality of the restaurant was that it was very inexpensive.
Quantity Star Rating: 0
Quantity Summary: 
Timeliness Star Rating: 0
Timeliness Summary: 



Kitchen Name: ARC KITCHEN
Quality Star Rating: 1
Quality Summary: Food quality has been horrible.
Quantity Star Rating: 0
Quantity Summary: 
Timeliness Star Rating: 0
Timeliness Summary: 



Kitchen Name: ARYA KITCHEN
Quality Star Rating: 2
Quality Summary: High-quality chicken on the chicken Caesar salad. I was proven dead wrong by this sushi bar, not only because the quality is great, but the service is fast and the food, impeccable.
Quantity Star Rating: 0
Quantity Summary: 
Timeliness Star Rating: 0
Timeliness Summary: 



Kitchen Name: KISHORE KITCHEN
Quality Star Rating: 0
Quality Summary: 
Quantity Star Rating: 0
Quantity Summary: 
Timeliness Star Rating: 0
Timeliness Summary: 



Kitchen Name: LOVERS KITCHEN
Quality Star Rating: 0
Quality Su

**Final code of the project**

In [ ]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

# Function to perform sentiment analysis on a review
def analyze_sentiment(review):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = analyzer.polarity_scores(review)
    # Return the compound score
    return sentiment_scores['compound']

# Function for in-line summarization
def generate_summary(reviews, sentences_count=6):
    # Concatenate reviews into a single document
    document = "\n".join(reviews)
    # Use sumy to parse the document and apply TextRank summarization
    parser = PlaintextParser.from_string(document, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    # Generate summary with specified number of sentences
    summary = summarizer(parser.document, sentences_count=sentences_count)
    # Convert the summary back to a string
    summary_text = " ".join(str(sentence) for sentence in summary)
    return summary_text

# Function to generate aspect-based star rating for quality, quantity, and timeliness
def generate_aspect_star_rating(reviews, aspect):
    # Calculate the average compound score for reviews mentioning the aspect
    aspect_reviews = [review for review in reviews if aspect.lower() in review.lower()]
    if not aspect_reviews:
        return 0
    aspect_compound_scores = [analyze_sentiment(review) for review in aspect_reviews]
    aspect_average_score = sum(aspect_compound_scores) / len(aspect_compound_scores)
    # Assign stars based on the average compound score
    if aspect_average_score >= 0.5:
        stars = 5  # Best, assign 5 stars
    elif aspect_average_score >= 0.4:
        stars = 4  # More than average, assign 4 stars
    elif aspect_average_score >= 0.1:
        stars = 3  # Neutral, assign 3 stars
    elif aspect_average_score >= -0.3:
        stars = 2  # Below average, assign 2 stars
    else:
        stars = 1  # Critical, assign 1 star
    return stars

# Load the CSV file into a DataFrame
df = pd.read_csv("Restaurant_Reviews.csv")

# Iterate through each kitchen in the DataFrame
for kitchen_name, group_df in df.groupby('Kitchen'):
    # Extract reviews for the current kitchen
    reviews = group_df['Review'].tolist()

    # Generate aspect-based star ratings and summaries
    quality_stars = generate_aspect_star_rating(reviews, 'quality')
    quantity_stars = generate_aspect_star_rating(reviews, 'quantity')
    timeliness_stars = generate_aspect_star_rating(reviews, 'timeliness')

    quality_summary = generate_summary([review for review in reviews if 'quality' in review.lower()])
    quantity_summary = generate_summary([review for review in reviews if 'quantity' in review.lower()])
    timeliness_summary = generate_summary([review for review in reviews if 'timeliness' in review.lower()])

    # Generate overall summary for all reviews
    overall_summary = generate_summary(reviews, sentences_count=10)  # Adjust sentences_count as needed

    # Print the results for the current kitchen
    print(f"\nKitchen Name: {kitchen_name}")
    print("Quality Star Rating:", quality_stars)
    print("Quantity Star Rating:", quantity_stars)
    print("Timeliness Star Rating:", timeliness_stars)

    print("Quality Summary:", quality_summary)
    print("Quantity Summary:", quantity_summary)
    print("Timeliness Summary:", timeliness_summary)

    print("Overall Summary:", overall_summary)
    print("\n" + "="*50 + "\n")



Kitchen Name: ANU'S KITCHEN
Quality Star Rating: 2
Quantity Star Rating: 0
Timeliness Star Rating: 0
Quality Summary: The only redeeming quality of the restaurant was that it was very inexpensive.
Quantity Summary: 
Timeliness Summary: 
Overall Summary: The selection on the menu was great and so were the prices. The only redeeming quality of the restaurant was that it was very inexpensive. Poor service, the waiter made me feel like I was stupid every time he came to the table. The only thing I did like was the prime rib and dessert section. My side Greek salad with the Greek dressing was so tasty, and the pita and hummus was very refreshing. We ordered the duck rare and it was pink and tender on the inside with a nice char on the outside. The salad had just the right amount of sauce to not over power the scallop, which was perfectly cooked. Update.....went back for a second time and it was still just as amazing We got the food and apparently they have never heard of salt and the batte